INSTALL THE FOLLOWING PYTHON PACKAGES FIRST BEFORE RUNNING THE PROGRAM

1) Numpy
2) NNFS - for the Spiral dataset
3) scikit-learn - for the iris dataset

In [242]:
# Library imports
import numpy as np

Create classes for modularity

In [243]:
# Hidden Layers
# Dense
class Layer_Dense:
    # Layer initialization
    # randomly initialize weights and set biases to zero
    def __init__(self, n_inputs, n_neurons):
        self.weights = np.random.randn(n_inputs, n_neurons) * np.sqrt(2. / n_inputs)
        self.biases = np.zeros((1, n_neurons))


    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)


In [244]:
# Activation Functions
# Included here are the functions for both the forward and backward pass

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)


class Activation_Softmax_Loss_CategoricalCrossentropy:
    def __init__(self):
        self.activation = Activation_Softmax()
        self.loss = Loss_CategoricalCrossEntropy()

    def forward(self, inputs, y_true):
        self.activation.forward(inputs)
        return self.loss.forward(self.activation.output, y_true)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        self.dinputs = dvalues.copy()
        self.dinputs[range(samples), y_true] -= 1
        self.dinputs = self.dinputs / samples


In [245]:
# Base Loss class
class Loss:
    # Computes the average loss over all samples
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        return np.mean(sample_losses)


# Mean Squared Error (MSE) Loss
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Compute squared difference and average per sample
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        # Derivative of MSE with respect to predictions
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize over number of samples
        self.dinputs /= samples


# Binary Cross-Entropy (BCE) Loss
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions to avoid log(0)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Compute BCE per sample
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        samples = y_true.shape[0]
        # Clip again to avoid division by 0
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Derivative of BCE
        self.dinputs = -(y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        self.dinputs /= samples

class Loss_CategoricalCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        samples = y_pred.shape[0]
        # Clip predictions to avoid log(0)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Select the predicted probability for the true class
        if len(y_true.shape) == 1:  # Sparse labels
            correct_confidences = y_pred_clipped[range(samples), y_true]
        else:  # One-hot encoded labels
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Compute negative log likelihood
        return -np.log(correct_confidences)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        labels = dvalues.shape[1]

        # Clip values to avoid division by zero
        dvalues_clipped = np.clip(dvalues, 1e-7, 1 - 1e-7)

        # Convert sparse labels to one-hot if needed
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Gradient
        self.dinputs = -y_true / dvalues_clipped
        self.dinputs /= samples


<!-- Star -->

In [246]:
class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0, adaptive=False, epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_lr = learning_rate
        self.decay = decay
        self.momentum = momentum
        self.adaptive = adaptive
        self.epsilon = epsilon
        self.iterations = 0

    def pre_update_params(self):
        # Apply learning rate decay
        if self.decay:
            self.current_lr = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))

    def update_params(self, layer):
        # ----- MOMENTUM -----
        if self.momentum:
            # Initialize momentum arrays if not present
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

            # Update momentum with current gradients
            weight_updates = self.momentum * layer.weight_momentums - self.current_lr * layer.dweights
            bias_updates = self.momentum * layer.bias_momentums - self.current_lr * layer.dbiases

            # Store momentum for next iteration
            layer.weight_momentums = weight_updates
            layer.bias_momentums = bias_updates
        else:
            # Vanilla SGD update (no momentum)
            weight_updates = -self.current_lr * layer.dweights
            bias_updates = -self.current_lr * layer.dbiases

        # ----- ADAPTIVE GRADIENT (Adagrad) -----
        if self.adaptive:
            if not hasattr(layer, 'weight_cache'):
                layer.weight_cache = np.zeros_like(layer.weights)
                layer.bias_cache = np.zeros_like(layer.biases)

            # Accumulate squared gradients
            layer.weight_cache += layer.dweights**2
            layer.bias_cache += layer.dbiases**2

            # Adjust updates by cached values
            weight_updates /= (np.sqrt(layer.weight_cache) + self.epsilon)
            bias_updates /= (np.sqrt(layer.bias_cache) + self.epsilon)

        # Apply updates
        layer.weights += weight_updates
        layer.biases += bias_updates

    def post_update_params(self):
        self.iterations += 1


Use most of the classes to create a functioning neural network, capable of performing a forward and backward pass

We can use a sample dataset from the Spiral module.  

We can also use the IRIS dataset.

In [247]:
# Spiral Data
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples = 100, classes = 3)

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [248]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [249]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris.data  # shape (150, 4)
y = iris.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [250]:
# ----- FIRST HIDDEN LAYER -----
dense1 = Layer_Dense(4, 8)
# This creates the first dense (fully connected) layer.
# It takes 4 input features (from your dataset) and produces 8 outputs (neurons).
# Each neuron learns a weighted combination of the 4 input values.

activation1 = Activation_ReLU()
# Applies the ReLU (Rectified Linear Unit) activation function.
# ReLU introduces non-linearity, helping the model learn complex relationships
# that a simple linear model couldn’t capture.

# ----- SECOND HIDDEN LAYER -----
dense2 = Layer_Dense(8, 8)
# Takes the 8 outputs from the previous layer and produces another 8 features.
# Adding more dense layers helps the network learn deeper representations
# (hierarchical features).

activation2 = Activation_ReLU()
# Again using ReLU to keep the model non-linear and prevent vanishing gradients.

# ----- OUTPUT LAYER -----
dense3 = Layer_Dense(8, 3)
# The final dense layer produces 3 outputs — one for each class in a classification task.

# ----- COMBINED SOFTMAX + LOSS FUNCTION -----
loss_activation = Activation_Softmax_Loss_CategoricalCrossentropy()
# This combines the Softmax activation (to convert the output scores into probabilities)
# with the Categorical Cross-Entropy loss function (to measure prediction error).
# Using this combined version is more numerically stable and efficient than
# running them separately.

# ----- OPTIMIZER CONFIGURATION -----
# --- First Run: Plain SGD ---
optimizer = Optimizer_SGD(learning_rate=0.1, decay=1e-3, momentum=0.0)
# Stochastic Gradient Descent (SGD) updates weights in the direction of the gradient.
# With momentum=0.0, it's the “vanilla” version — can be slower and noisier,
# but it’s good for comparison.

# --- Second Run: SGD + Momentum ---
# optimizer = Optimizer_SGD(learning_rate=0.1, decay=1e-3, momentum=0.9)
# The momentum version remembers past gradients and smooths updates.
# It helps escape local minima and typically converges faster with more stable loss.


PERFORM ONLY 1 PASS

In [251]:
EPOCHS = 1000

for i in range(EPOCHS):
    # Pre-update step (handles learning rate decay)
    optimizer.pre_update_params()

    # ----- Forward Pass -----
    dense1.forward(X)
    activation1.forward(dense1.output)

    dense2.forward(activation1.output)
    activation2.forward(dense2.output)

    dense3.forward(activation2.output)

    # Combined Softmax + Loss in one step
    loss = loss_activation.forward(dense3.output, y)
    predictions = np.argmax(loss_activation.activation.output, axis=1)
    accuracy = np.mean(predictions == y)

    # Log progress
    if i % 100 == 0:
        print(f"Epoch {i}, Loss: {np.mean(loss):.3f}, Acc: {accuracy:.3f}")

    # ----- Backward Pass -----
    # Backprop from Softmax + CCE combined
    loss_activation.backward(loss_activation.activation.output, y)
    dense3.backward(loss_activation.dinputs)
    activation2.backward(dense3.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # ----- Update Weights -----
    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    optimizer.update_params(dense3)
    optimizer.post_update_params()

# After training
print("\nFinal comparison:")
print("Predictions:", predictions[:10])
print("Actual:     ", y[:10])


Epoch 0, Loss: 0.903, Acc: 0.593
Epoch 100, Loss: 0.303, Acc: 0.947
Epoch 200, Loss: 0.170, Acc: 0.967
Epoch 300, Loss: 0.116, Acc: 0.973
Epoch 400, Loss: 0.091, Acc: 0.980
Epoch 500, Loss: 0.077, Acc: 0.980
Epoch 600, Loss: 0.069, Acc: 0.980
Epoch 700, Loss: 0.063, Acc: 0.987
Epoch 800, Loss: 0.059, Acc: 0.987
Epoch 900, Loss: 0.056, Acc: 0.987

Final comparison:
Predictions: [0 0 0 0 0 0 0 0 0 0]
Actual:      [0 0 0 0 0 0 0 0 0 0]


In [252]:
# Perform a backward pass of our training data
# From loss to 2nd softmax activation
loss_function.backward(activation2.output, y)
dvalues = loss_function.dinputs # Gradient of the loss w.r.t softmax output

print(dvalues.shape)
# print(dvalues)

# From 2nd softmax to 2nd dense layer
activation2.backward(dvalues)
# From 2nd dense layer to 1st ReLU activation
dense2.backward(activation2.dinputs)

# From 1st ReLU activation to 1st dense layer
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

(150, 8)


/tmp/ipykernel_17543/4089102767.py:81: RuntimeWarning: divide by zero encountered in divide
  self.dinputs = -y_true / dvalues
/tmp/ipykernel_17543/4089102767.py:81: RuntimeWarning: invalid value encountered in divide
  self.dinputs = -y_true / dvalues


In [253]:
# Check the gradient values of the weights and biases of the established layers
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)


# Update the weights and biases
optimizer.update_params(dense1)
optimizer.update_params(dense2)

[[-0.03742512 -0.02503555 -0.97576136 -0.54345757  0.4956472   0.32721303
   0.15203753  0.47787319]
 [-0.05276055  0.14791076  0.1325332   0.11814028 -0.04274488 -0.05922996
   0.03849154 -0.13584872]
 [-0.01653393 -0.04651052 -0.6713811  -0.40827398  0.34629214  0.24350544
   0.07811247  0.3679393 ]
 [-0.00589372 -0.02224812 -0.47124062 -0.28613283  0.21360845  0.17152169
   0.09194329  0.25724844]]
[[ 0.04026714 -0.50836703 -0.89743878 -0.35406682  0.37552392  0.17207583
  -0.05747556  0.20581067]]
[[ 0.         -0.02700632  0.          0.          0.          0.
   0.          0.        ]
 [ 0.         -1.7988825  -1.23890233  0.          0.          0.
   0.          0.        ]
 [ 0.         -0.34509055 -0.72941775  0.          0.          0.
   0.          0.        ]
 [ 0.         -2.07496145 -1.16168039  0.          0.          0.
   0.          0.        ]
 [ 0.         -0.32053136 -0.16810948  0.          0.          0.
   0.          0.        ]
 [ 0.         -0.25144716 -0